In [7]:
import numpy as np
from scipy import linalg
from scipy.sparse import diags, csc_matrix, kron
from scipy.sparse import linalg as spla
import matplotlib.pyplot as plt
from pathlib import Path

## Helper functions

In [9]:
def printmatrix(mat, **kwargs):
    fp = kwargs.get("fp", 3)    # number of floating points
    bs = kwargs.get("bs", 8)   # number of blank-spaces
    format_dict = {
        'float_kind': lambda x: f"{x:>{bs}.{fp}f}",
        'bool': lambda x: False if x == 0 else True,
        }
    formatter = kwargs.get("formatter", format_dict)
    linewidth = kwargs.get("linewidth", None)
    if linewidth is None:
        linewidth = kwargs.get("lw", 100)
    with np.printoptions(formatter=formatter, linewidth=linewidth):
        print(mat)

## Create folder for figures

In [11]:
# make directory if not already present
assignment_dir = Path.cwd() # directory for this working directory (if using notebook, it will be the direcotry of notebook)
figures_dir = assignment_dir / "figures" # figures directory
figures_dir.mkdir(parents=True, exist_ok=True) # create folder and parent directories, if not already existing.

# Exercise 1 Design ODE solver

## Subtask 1.1)